In [1]:
import altair as alt
from altair import datum

from src.data.read_dataset import get_processed_dataset
from src.visualization import visualize as viz

import ipywidgets as widgets
from ipywidgets import interact, interact_manual
from IPython.display import HTML, Markdown as md

In [2]:
regione = 5

In [3]:
regioni_df = get_processed_dataset('dpc_regioni')
prov_df = get_processed_dataset('dpc_province')

reg_df = regioni_df[regioni_df.codice_regione == regione]
prov_df = prov_df[(prov_df.codice_regione == regione) & (~prov_df.provincia.str.contains('definizione'))]
today = reg_df.iloc[-1]
yesterday = reg_df.iloc[-2]

In [4]:
md(f"# Riepilogo regione {reg_df.regione.iloc[0]}")

# Riepilogo regione Veneto

In [6]:
html = viz.tabella_riassuntiva(today, yesterday)
HTML(html)

,Oggi,Variazione
Totale Casi,19174,6
Totale Positivi,1229,-90
Tamponi,713500,7025
Casi Testati,346872.0,2377
Deceduti,1938,4
Dimessi Guariti,16007,92
Terapia Intensiva,2,0
Incremento,0.03%,0.01
Percentuale Positivi,5.53%,-0.04
Percentuale Nuovi Positivi,0.25%,0.13


In [7]:
viz.grafici_riassuntivi(reg_df)

alt.VConcatChart(...)

In [8]:
c = alt.Chart(prov_df).mark_line().encode(
        x = 'data:T',
        color = 'provincia:N',
        tooltip = ['data', 'provincia', 'totale_casi', 'nuovi_positivi', 'incremento']
    ).properties(
        width=800,
        height=600
    ).interactive()

@interact(logarithm=False)
def plot_province(logarithm):    
    if logarithm:
        return c.encode(y = alt.Y('totale_casi:Q', scale=alt.Scale(type='log'))).transform_filter(datum.totale_casi > 100)
    else:
        return c.encode(y = alt.Y('totale_casi:Q')).transform_filter(datum.totale_casi > 100)

interactive(children=(Checkbox(value=False, description='logarithm'), Output()), _dom_classes=('widget-interac…

In [9]:
c.encode(y = alt.Y(f'incremento:Q', scale=alt.Scale(domain=(0, 10))), row='provincia').properties(
    width=800,
    height=100
)

alt.Chart(...)

In [10]:
latest = prov_df.groupby('provincia').last().reset_index()

alt.Chart(latest).mark_bar().encode(
    x='nuovi_positivi:Q',
    y=alt.Y('provincia:N', sort='-x'),
).properties(
    height=300,
    width=800,
    title=f'Nuovi casi odierni per provincia, data = {latest.iloc[0, 1]}'
).interactive()


alt.Chart(...)

In [11]:
base = alt.Chart().transform_window(
    averaged='mean(nuovi_positivi)',
    frame=[-4,3],
    groupby=['provincia']
).encode(
    x='data:T',
    tooltip=['data', 'provincia', 'totale_casi', 'nuovi_positivi', 'incremento']
).properties(
    height=200,
    width=200,
)

c1 = base.mark_line().encode(
    y=alt.Y('averaged:Q', title=''),
)

c2 = base.mark_bar(opacity=0.2, width=5).encode(
    y=alt.Y('nuovi_positivi', title=''),

)

alt.layer(c1, c2, data=prov_df).facet(facet='provincia:N', columns=4).properties(
    title={
      "text": ['Nuovi casi per provincia'], 
      "subtitle": ["Dato mediato su 7 giorni"]
    }
).interactive()

alt.FacetChart(...)

In [12]:
base = alt.Chart().transform_window(
    averaged='mean(nuovi_positivi_per_1M_pop)',
    frame=[-4,3],
    groupby=['provincia']
).encode(
    x='data:T',
    tooltip=['data', 'provincia', 'totale_casi', 'nuovi_positivi', 'incremento']
).properties(
    height=200,
    width=200,
)

c1 = base.mark_line().encode(
    y=alt.Y('averaged:Q', title=''),
)

c2 = base.mark_bar(opacity=0.1).encode(
    y=alt.Y('nuovi_positivi_per_1M_pop', title=''),

)

alt.layer(c1, c2, data=prov_df).facet(facet='provincia:N', columns=4).properties(
    title={
      "text": ['Nuovi casi per 1M di abitanti per provincia'], 
      "subtitle": ["Dato mediato su 7 giorni"]
    }
).interactive()

alt.FacetChart(...)

In [13]:
latest = prov_df.groupby('provincia').last().reset_index()

alt.Chart(latest).mark_bar().encode(
    x='totale_casi:Q',
    y=alt.Y('provincia:N', sort='-x'),
).properties(
    height=300,
    width=800,
    title='Totale casi per provincia'
) & alt.Chart(latest).mark_bar().encode(
    x='totale_casi_per_1M_pop:Q',
    y=alt.Y('provincia:N', sort='-x'),
).properties(
    height=300,
    width=800,
    title='Totale casi per 1M abitanti per provincia'
).interactive()

alt.VConcatChart(...)